In [1]:

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)


import os


In [2]:
import zipfile

with zipfile.ZipFile("/kaggle/input/ada511-training-transducer-dataset/test.zip","r") as z:
    z.extractall(".")
    

In [3]:
import fastai

In [4]:
test = "/kaggle/working/test"

In [5]:


from fastai.vision.all import *

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [6]:
learn = load_learner('/kaggle/input/ada511-training-transducer-dataset/resnet18.pkl')

In [7]:
# Initialize DataFrame
test_df = pd.DataFrame(columns=['Predicted', 'Actual'])
from pathlib import Path

test_folder = Path('/kaggle/working/test')
# Function to process each image
def process_image(image_path, label):
    img_fastai = load_image(image_path)
    pred_class, pred_idx, outputs = learn.predict(img_fastai)
    return {'Predicted': pred_class, 'Actual': label}

# Iterate over each subfolder ('cat' and 'dog') and process images
results = []
for label in ['cat', 'dog']:
    subfolder_path = test_folder / label
    if subfolder_path.exists() and subfolder_path.is_dir():
        for image_file in subfolder_path.iterdir():
            if image_file.is_file():
                result = process_image(image_file, label)
                results.append(result)
test_df = pd.DataFrame(results)

In [8]:
# Initialize the confusion matrix DataFrame
cd_df = pd.DataFrame(columns=['True Cat', 'True Dog'], index=['Predicted Cat', 'Predicted Dog'])

# Initialize counters
true_cat_as_cat = 0
true_dog_as_dog = 0
cat_as_dog = 0
dog_as_cat = 0

# Iterate through the test_df to count instances
for index, row in test_df.iterrows():
    if row['Actual'] == 'cat' and row['Predicted'] == 'cat':
        true_cat_as_cat += 1
    elif row['Actual'] == 'dog' and row['Predicted'] == 'dog':
        true_dog_as_dog += 1
    elif row['Actual'] == 'cat' and row['Predicted'] == 'dog':
        cat_as_dog += 1
    elif row['Actual'] == 'dog' and row['Predicted'] == 'cat':
        dog_as_cat += 1

# Fill the DataFrame
cd_df.loc['Predicted Cat', 'True Cat'] = true_cat_as_cat
cd_df.loc['Predicted Cat', 'True Dog'] = dog_as_cat
cd_df.loc['Predicted Dog', 'True Cat'] = cat_as_dog
cd_df.loc['Predicted Dog', 'True Dog'] = true_dog_as_dog

# Display the confusion matrix
print(cd_df)


              True Cat True Dog
Predicted Cat     1744       38
Predicted Dog        6     1712


In [9]:
test_df.to_csv('test_dataframe.csv', index=False)

In [10]:
cd_df.to_csv('cd_dataframe.csv')